### **SQLAlchemy를 활용한 채팅 기록 저장**


- `SQL (Structured Query Language)`: 관계형 데이터베이스(`RDBMS`)에서 데이터를 관리하는 언어

- `SQLAlchemy`: `Python용` 오픈 소스 SQL 툴킷 및 ORM(Object-Relational Mapper).

- `SQLChatMessageHistory`: `SQLAlchemy를` 이용해 채팅 기록을 저장하는 클래스.

- `SQLite` 외의 데이터베이스를 사용하려면 해당 DB 드라이버 설치 필요.


**db파일에 히스토리를 저장하여 사용자가 채팅을 떠나도 대화 내용을 기억 할 수 있게 함**

### **사용방법**


1. `session_id` - 사용자 이름, 이메일, 채팅 ID 등과 같은 세션의 고유 식별자.

2. `connection` - 데이터베이스 연결을 지정하는 문자열, 이 문자열은 SQLAlchemy의 create_engine 함수에 전달


In [1]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 파일을 설정
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite_test.db"
)

In [2]:
# 사용자 메시지를 추가합니다.
chat_message_history.add_user_message(
    "안녕 나는 AI개발을 하고있어 회사를 다니는 동시에 대학원에서 공부중이야"
)
# AI 메시지를 추가합니다.
chat_message_history.add_ai_message("안녕 반가워 나도 잘 부탁해")

- 저장된 대화내용을 확인합니다. `chat_message_history.messages`


In [ ]:
# 채팅 메시지 히스토리리
chat_message_history.messages

[HumanMessage(content='안녕 나는 AI개발을 하고있어 회사를 다니는 동시에 대학원에서 공부중이야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 반가워 나도 잘 부탁해', additional_kwargs={}, response_metadata={})]

### **Chain 에 적용**

메세지를  `LCEL Runnables` 와 결합

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지
        ("system", "You are a helpful assistant."),
        # 대화 기록을 위한 Placeholder
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),  # 질문
    ]
)

# chain 을 생성합니다.
chain = prompt | ChatOpenAI(model_name="gpt-4o-mini") | StrOutputParser()

In [9]:
# sqlite db 에서 대화내용을 가져오는 함수
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite_test.db",
    )

`config_fields` 를 설정합니다. 이는 대화정보를 조회할 때 참고 정보로 활용합니다.

- `user_id`: 사용자 ID
- `conversation_id`: 대화 ID

In [ ]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,  # 대화 기록을 가져오는 함수를 설정합니다.
    input_messages_key="question",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="chat_history",  # 대화 기록 메시지의 키를 "history"로 설정
    history_factory_config=config_fields,  # 대화 기록 조회시 참고할 파라미터를 설정합니다.
)

- `"configurable"` 키 아래에 `"user_id"`, `"conversation_id"` key-value 쌍을 설정합니다.


In [ ]:
# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

질문에 이름을 물어보는 질문을 해보겠습니다. 이전에 저장한 대화가 있다면, 올바르게 답할 것입니다.

- `chain_with_history` 객체의 `invoke` 메서드를 호출하여 질문에 대한 답변을 생성합니다.
- `invoke` 메서드에는 질문 딕셔너리와 `config` 설정이 전달됩니다.


In [ ]:
# 질문과 config 를 전달하여 실행합니다.
chain_with_history.invoke({"question": "안녕 반가워, 내 이름은 테디야"}, config)

In [ ]:
# 후속 질문을 실해합니다.
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

이번에는 같은 `user_id` 를 가지지만 `conversion_id` 가 다른 값을 가지도록 설정합니다.

In [ ]:
# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation2"}}

# 질문과 config 를 전달하여 실행합니다.
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)